<a href="https://colab.research.google.com/github/wujulie/ML__homework/blob/main/homework_2%E4%BF%AE%E6%AD%A3%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
data=pd.read_csv("/content/drive/My Drive/bank.csv",delimiter=";",header='infer')
pd.set_option('display.max_columns',None)
print(data)
train_data=pd.DataFrame(data)
print(train_data.shape)
train_data['job'].replace(['unemployed', 'services', 'management', 'blue-collar',
                     'self-employed', 'technician', 'entrepreneur', 'admin.', 'student',
                     'housemaid', 'retired', 'unknown'],[i for i in range (1,data['job'].unique().size+1)],inplace=True)

train_data.marital.replace(['married','single','divorced'],[0,1,2],inplace=True)
train_data.education.replace(['primary','secondary','tertiary','unknown'],[0,1,2,3],inplace=True)
train_data.default.replace(['yes','no'],[1,0],inplace=True) #信用程度
train_data.housing.replace(['yes','no'],[1,0],inplace=True)
train_data.loan.replace(['yes','no'],[1,0],inplace=True)#借貸
train_data.contact.replace(['cellular','telephone','unknown'],[0,1,2],inplace=True)#借貸
train_data.month.replace(['jun','feb','mar','apr','may','jan','jul','aug',
                         'sep','oct','nov','dec'],[1,2,3,4,5,6,7,8,9,10,11,12],inplace=True)
train_data.poutcome.replace(['failure','success','unknown','other'],[0,1,2,3],inplace=True)
train_data.y.replace(['no','yes'],[0,1],inplace=True)

      age            job  marital  education default  balance housing loan  \
0      30     unemployed  married    primary      no     1787      no   no   
1      33       services  married  secondary      no     4789     yes  yes   
2      35     management   single   tertiary      no     1350     yes   no   
3      30     management  married   tertiary      no     1476     yes  yes   
4      59    blue-collar  married  secondary      no        0     yes   no   
...   ...            ...      ...        ...     ...      ...     ...  ...   
4516   33       services  married  secondary      no     -333     yes   no   
4517   57  self-employed  married   tertiary     yes    -3313     yes  yes   
4518   57     technician  married  secondary      no      295      no   no   
4519   28    blue-collar  married  secondary      no     1137      no   no   
4520   44   entrepreneur   single   tertiary      no     1136     yes  yes   

       contact  day month  duration  campaign  pdays  previous 

In [ ]:
bank_data=train_data.iloc[:,:-1]#X
target=train_data.iloc[:,-1]#Y

In [ ]:
#Feature Importance Filter 
from sklearn.feature_selection import SelectKBest,f_regression
kb_regr = SelectKBest(f_regression)
x_b = kb_regr.fit_transform(bank_data,target)
print(x_b.shape,'\n',kb_regr.scores_)

(4521, 10) 
 [9.20707070e+00 2.16039030e+01 1.70127936e+01 8.36596827e+00
 7.66832180e-03 1.44922234e+00 5.00706834e+01 2.25837165e+01
 8.21193103e+01 5.71419411e-01 3.13973921e+00 8.66505760e+02
 1.69599922e+01 4.94953804e+01 6.24091442e+01 2.03459313e+01]


In [ ]:
#Low Variance Filter
from sklearn.preprocessing import Normalizer 
no = Normalizer()  ##預設是 l2 norm 
scaled_data = no.fit_transform(bank_data)
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=1e-05) 
X_t = vt.fit_transform(scaled_data) 
print(X_t.var(axis=0))

[1.53867915e-02 3.04079525e-04 2.37080805e-05 2.56794927e-01
 1.74531737e-05 3.70103089e-03 4.16172496e-04 1.18588375e-01
 4.87814010e-04 3.21154947e-02 1.36052901e-05 3.69474281e-05]


In [ ]:
#Principal component analysis 
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(bank_data)
print(X_std.var(axis=1))
from sklearn.decomposition import PCA
pca = PCA(n_components=12)
x_pca = pca.fit_transform(X_std)
print(x_pca.shape,X_std.shape)
print(pca.explained_variance_ratio_)

[0.58578882 1.9665417  1.26481497 ... 0.83652212 0.9773403  1.69366838]
(4521, 12) (4521, 16)
[0.13797187 0.10364887 0.08959993 0.07412694 0.07204217 0.06453454
 0.06088485 0.05760934 0.05691128 0.05537427 0.05034637 0.04641891]


In [ ]:
#High Correlation Filter
import pandas as pd 
df = pd.DataFrame(X_std) 
print(df.corr())
X_h = df.drop([13,14],axis=1)
df = pd.DataFrame(X_h) 
print(X_h.shape,'\n',df.corr())

          0         1         2         3         4         5         6   \
0   1.000000  0.252828 -0.123804 -0.121613 -0.017885  0.083820 -0.193888   
1   0.252828  1.000000 -0.012186 -0.088132 -0.002168  0.045592 -0.160040   
2  -0.123804 -0.012186  1.000000  0.090907  0.049837 -0.028742 -0.027397   
3  -0.121613 -0.088132  0.090907  1.000000 -0.010534  0.057725 -0.087070   
4  -0.017885 -0.002168  0.049837 -0.010534  1.000000 -0.070886  0.006881   
5   0.083820  0.045592 -0.028742  0.057725 -0.070886  1.000000 -0.050227   
6  -0.193888 -0.160040 -0.027397 -0.087070  0.006881 -0.050227  1.000000   
7  -0.011250  0.001375 -0.014321 -0.054086  0.063994 -0.071349  0.018451   
8   0.015161 -0.037670 -0.021157 -0.110554  0.008448 -0.009665  0.196454   
9  -0.017853  0.000306 -0.002284  0.014926 -0.013261 -0.008677 -0.031291   
10  0.045217  0.029440 -0.016776  0.070070  0.002794  0.044242 -0.115831   
11 -0.002367 -0.012931  0.035321 -0.014878 -0.011615 -0.015950  0.015740   
12 -0.005148

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
def model(x,y):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=45)
  model = LinearRegression() 
  model.fit(X_train, y_train) 
  y_pred = model.predict(X_test)
  print('Y_pred', y_pred)
  index = 0
  for i in y_pred:
    if i >0.5:
      y_pred[index]=1
    else:
      y_pred[index]=0
    index+=1
  print('classfy y_pred: ', y_pred)
  print('accuracy:', accuracy_score(y_test,y_pred))

  # model = LogisticRegression() 
  # model.fit(X_train, y_train) 
  # y_pred = model.predict(X_test)
  # print('Score: ', model.score(X_train, y_train))

In [ ]:
print('---use Feature Importance Filter ---')
model(x_b,target)
print('\n---use High Correlation Filter---')
model(X_h,target)
print('\n---use Low Variance Filter---')
model(X_t,target)
print('\n---use PCA---')
model(x_pca,target)

---use Feature Importance Filter ---
Y_pred [0.21863412 0.04234376 0.14245629 ... 0.02876699 0.10438487 0.0051377 ]
classfy y_pred:  [0. 0. 0. ... 0. 0. 0.]
accuracy: 0.8960943257184967

---use High Correlation Filter---
Y_pred [ 0.17527142  0.01799042  0.14133039 ...  0.08006406  0.09536345
 -0.00207747]
classfy y_pred:  [0. 0. 0. ... 0. 0. 0.]
accuracy: 0.8975681650700074

---use Low Variance Filter---
Y_pred [0.14787215 0.0743307  0.18246275 ... 0.09592005 0.16935807 0.14435122]
classfy y_pred:  [0. 0. 0. ... 0. 0. 0.]
accuracy: 0.887988209285188

---use PCA---
Y_pred [ 0.22026592  0.05926607  0.12908996 ...  0.07597692  0.06570836
 -0.00584116]
classfy y_pred:  [0. 0. 0. ... 0. 0. 0.]
accuracy: 0.899042004421518


NameError: ignored